In [ ]:
# -m pip install networkx
# -m pip install matplotlib
# !pip install tqdm
# !pip install pandas
# !pip install numpy
# !pip install graphviz
# !pip install scikit-learn

In [80]:
import random
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import combinations, groupby

from networkx.algorithms import tree
from networkx.algorithms import bellman_ford_predecessor_and_distance
from networkx.algorithms import floyd_warshall_predecessor_and_distance

import numpy.typing as npt

# Task 1. Algorithm's analysis

## Generating graph

In [81]:

# You can use this function to generate a random graph with 'num_of_nodes' nodes
# and 'completeness' probability of an edge between any two nodes
# If 'directed' is True, the graph will be directed
# If 'draw' is True, the graph will be drawn
def gnp_random_connected_graph(num_of_nodes: int,
                               completeness: int,
                               directed: bool = False,
                               draw: bool = False):
    """
    Generates a random graph, similarly to an Erdős-Rényi 
    graph, but enforcing that the resulting graph is conneted (in case of undirected graphs)
    """

    
    if directed:
        G = nx.DiGraph()
    else:
        G = nx.Graph()
    edges = combinations(range(num_of_nodes), 2)
    G.add_nodes_from(range(num_of_nodes))
    
    for _, node_edges in groupby(edges, key = lambda x: x[0]):
        node_edges = list(node_edges)
        random_edge = random.choice(node_edges)
        if random.random() < 0.5:
            random_edge = random_edge[::-1]
        G.add_edge(*random_edge)
        for e in node_edges:
            if random.random() < completeness:
                G.add_edge(*e)
                
    for (u,v,w) in G.edges(data=True):
        w['weight'] = random.randint(-5, 20)
                
    if draw: 
        plt.figure(figsize=(10,6))
        if directed:
            # draw with edge weights
            pos = nx.arf_layout(G)
            nx.draw(G,pos, node_color='lightblue', 
                    with_labels=True,
                    node_size=500, 
                    arrowsize=20, 
                    arrows=True)
            labels = nx.get_edge_attributes(G,'weight')
            nx.draw_networkx_edge_labels(G, pos,edge_labels=labels)
            
        else:
            nx.draw(G, node_color='lightblue', 
                with_labels=True, 
                node_size=500)
        
    return G

In [ ]:
G = gnp_random_connected_graph(5, 5, True, True)

## Subtask 1.1

### Kruskal's algorithm

In [ ]:
mstk = tree.minimum_spanning_tree(G, algorithm="kruskal")

In [ ]:
nx.draw(mstk, node_color='lightblue', 
        with_labels=True, 
        node_size=500)

In [ ]:
mstk.edges(), len(mstk.edges())

*put your code below* (delete this)

### Prim's algorithm

In [8]:
mstp = tree.minimum_spanning_tree(G, algorithm="prim")

In [ ]:
nx.draw(mstp, node_color='lightblue', 
        with_labels=True, 
        node_size=500)

In [ ]:
mstp.edges(), len(mstp.edges())

In [ ]:
def prim(graph):
    '''
    computes prim's algorythm
    '''
    
    start = random.choice(list(graph.nodes))
    mst = nx.Graph()
    visited = {start}

    while len(visited) < len(graph.nodes):
        best_weight = float('inf')
        best_edge = None

        for node in visited:
            for neighbor, info in graph[node].items():
                weight = info['weight']
                if neighbor not in visited and weight < best_weight:
                    best_weight = weight
                    best_edge = (node, neighbor)
        
        if best_edge:
            mst.add_edge(best_edge[0], best_edge[1], weight=best_weight)
            visited.add(best_edge[1])

    return mst

In [ ]:
num_of_nodes = 10
completeness = 0.5
G = gnp_random_connected_graph(num_of_nodes, completeness, directed=False, draw=False)
mst = prim(G)
nx.draw(mst, node_color='lightblue', with_labels=True, node_size=500) 

## Subtask 1.2

In [ ]:
G = gnp_random_connected_graph(10, 0.5, True, True)
mst = prim(G)
nx.draw(mst, node_color='lightblue', 
        with_labels=True,       
        node_size=500)

### Bellman-Ford algorithm

In [ ]:
""" belman ford algorithm """

def bellman_ford(graph: dict, source_vertex: str) -> dict[int | None]:
    """
    Takes in a graph, which is the graph in format (starting_vertex, ending_vertex, weight)
    also takes in the number of vertices and the source vertex

    returns a dict of the shortest distances or None if negative is detected

    Example:
        >>> import networkx as nx
        >>> G = nx.DiGraph()
        >>> G.add_weighted_edges_from([\
                (1, 2, 6),\
                (1, 5, 7),\
                (2, 5, 8),\
                (5, 4, 9),\
                (4, 3, 7),\
                (3, 2, -2),\
                (2, 3, 5),\
                (2, 4, -4),\
                (5, 3, -3)\
            ])
        >>> result = bellman_ford(G, 1)
        >>> print(result)
        Distance to 1: 0
        Distance to 2: 2
        Distance to 3: 4
        Distance to 4: -2
        Distance to 5: 7
    """
    # Make starting dictionary
    distances_dict = {node: float('inf') for node in graph.nodes}
    distances_dict[source_vertex] = 0
    num_vertices = len(graph.nodes)

    # Do algortihm vertices_num -1 times
    for _ in range(num_vertices - 1):
        for u, v, weights in graph.edges(data=True):
            weight = weights['weight']
            if distances_dict[u] + weight < distances_dict[v]:
                distances_dict[v] = distances_dict[u] + weight

    # Check extra time for neg cycle
    for u, v, weights in graph.edges(data=True):
        weight = weights['weight']
        if distances_dict[u] + weight < distances_dict[v]:
            print("The graph contains a negaitve weight cycle")

    # Sort starting dict
    sorted_dict = dict(sorted(distances_dict.items(), key=lambda item: item[0]))

    # Return nice output
    output = ''
    for num, (key, val) in enumerate(sorted_dict.items()):
        if num == len(sorted_dict) - 1:
            output += f"Distance to {key}: {val}"
            break
        output += f"Distance to {key}: {val}\n"
    return output

if __name__ == "__main__":
    import doctest
    print(doctest.testmod())


### Floyd-Warshall algorithm

In [ ]:
# pred is a dictionary of predecessors, dist is a dictionary of distances dictionaries
try:
    pred, dist = floyd_warshall_predecessor_and_distance(G) 
    for k, v in dist.items():
        print(f"Distances with {k} source:", dict(v))
except:
    print("Negative cycle detected")

*put your code below* (delete this)

---

## Some useful explanations
### How to get list of edges for your algorithm

In [14]:
edges = list(G.edges()) # by default G.edges are EdgesView class

In [ ]:
edges[:5]

### To get edges with weights

In [16]:
edges = list(G.edges(data=True))

In [ ]:
edges[:5]

In [ ]:
nodes = list(G.nodes())
print(nodes)

## Example on time measuring

Read more on this: https://realpython.com/python-timer/

Recall that you should measure times for 5, 10, 20, 50, 100, 200, 500 nodes 1000 times (and take mean of time taken for each node amount).

Then you should build the plot for two algorithms (x - data size, y - mean time of execution).

In [19]:
import time
from tqdm import tqdm

In [ ]:
NUM_OF_ITERATIONS = 1000
time_taken = 0
for i in tqdm(range(NUM_OF_ITERATIONS)):
    
    # note that we should not measure time of graph creation
    G = gnp_random_connected_graph(100, 0.4, False)
    
    start = time.time()
    tree.minimum_spanning_tree(G, algorithm="prim")
    end = time.time()
    
    time_taken += end - start

time_taken / NUM_OF_ITERATIONS

## Task 2. Decision Tree Classifier 

In [110]:
# scikit-learn package
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.model_selection import train_test_split

![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)

### Visualization of produced tree

You do not need to understand this piece of code :)

In [ ]:
import graphviz 
dot_data = tree.export_graphviz(clf, out_file=None) 
graph = graphviz.Source(dot_data) 
graph.render("iris")

In [ ]:
dot_data = tree.export_graphviz(clf, out_file=None, 
                     feature_names=iris.feature_names,  
                     class_names=iris.target_names,  
                     filled=True, rounded=True,  
                     special_characters=True)  
graph = graphviz.Source(dot_data)  
graph 

### Prediction step

Now we can use our model to predict which type has a flower, basing on its parameters.

This is conducted basically via traversing the tree that you can see above.

![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)

### Finally, it is your turn to write such classifier by yourself!

####  Gini impurity

Decision trees use the concept of Gini impurity to describe how “pure” a node is. A node is pure (G = 0) if all its samples belong to the same class, while a node with many samples from many different classes will have a Gini closer to 1.

$G = 1 - \sum_{k=1}^{n}p_{k}^2$

For example, if a node contains five samples, with two belonging to the first class (first flower), two of class 2, one of class 3 and none of class 4, then

$G = 1 - (\frac{2}{5})^2 - (\frac{2}{5})^2 - (\frac{1}{5})^2 = 0.64$

#### Remarks 
- We recommend using additional functions in `DecisionTreeClassifier` class, to make the implementation process easier.
- [use this hint](https://arc.net/l/quote/pqvyjqei)

In [ ]:
# importing all neccesary libraries
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from networkx.algorithms import tree

In [ ]:
# setting things up

data = pd.read_csv('drug200.csv')
encoding = {
    'Sex': {'F': 0, 'M': 1},
    'BP': {'LOW': 0, 'NORMAL': 1, 'HIGH': 2},
    'Cholesterol': {'NORMAL': 0, 'HIGH': 1},
    'Drug': {'drugA': 0, 'drugB': 1, 'drugC': 2, 'drugX': 3, 'drugY': 4}
}

data['Sex'] = data['Sex'].map(encoding['Sex'])
data['BP'] = data['BP'].map(encoding['BP'])
data['Cholesterol'] = data['Cholesterol'].map(encoding['Cholesterol'])
data['Drug'] = data['Drug'].map(encoding['Drug'])

X = data.drop('Drug', axis=1).values
y = data['Drug'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
# DECISION TREE

class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, label=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.label = label

class DecisionTree:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth
        self.root = None
    
    @staticmethod
    def gini(y):
        total_counter = len(y)
        classes_counter = {}
        for label in y:
            classes_counter[label] = classes_counter.get(label, 0) + 1
        impurity = 1 - sum((count / total_counter) ** 2 for count in classes_counter.values())
        return impurity
    
    def best_split(self, X, y):
        min_gini = float('inf')
        best_split = None
        best_left_idx, best_right_idx = None, None
        
        for feature_id in range(X.shape[1]):
            thresholds = np.unique(X[:, feature_id])
            for threshold in thresholds:
                left_idx = np.where(X[:, feature_id] <= threshold)[0]
                right_idx = np.where(X[:, feature_id] > threshold)[0]

                if left_idx.size == 0 or right_idx.size == 0:
                    continue

                left_y = y[left_idx]
                right_y = y[right_idx]

                gini_score = (len(left_y) * self.gini(left_y) + len(right_y) * self.gini(right_y)) / len(y)

                if gini_score < min_gini:
                    min_gini = gini_score
                    best_split = (feature_id, threshold)
                    best_left_idx = left_idx
                    best_right_idx = right_idx
        
        return best_split, best_left_idx, best_right_idx
    
    def build_tree(self, X, y, depth=0):
        if len(set(y)) == 1:
            return Node(label=y[0])

        if self.max_depth and depth >= self.max_depth:
            most_common_label = max(set(y), key=list(y).count)
            return Node(label=most_common_label)

        best_split, left_idx, right_idx = self.best_split(X, y)
        if not best_split:
            most_common_label = max(set(y), key=list(y).count)
            return Node(label=most_common_label)

        feature, threshold = best_split
        left_subtree = self.build_tree(X[left_idx], y[left_idx], depth + 1)
        right_subtree = self.build_tree(X[right_idx], y[right_idx], depth + 1)
        
        return Node(feature=feature, threshold=threshold, left=left_subtree, right=right_subtree)
    
    def fit(self, X, y):
        self.root = self.build_tree(X, y)

    def predict(self, X):
        predictions = []
        for row in X:
            node = self.root
            while node.label is None:
                if row[node.feature] <= node.threshold:
                    node = node.left
                else:
                    node = node.right
            predictions.append(node.label)
        return predictions

In [ ]:
# Initializing and training the decision tree
tree = DecisionTree(max_depth=0)
tree.fit(X_train, y_train)

# Making predictions
predictions = tree.predict(X_test)
print("Predictions:", predictions)
print(f"Accuracy = {sum(predictions == y_test) / len(y_test)}")

In [ ]:
# visualization 
import graphviz 
dot_data = tree.export_graphviz(clf, out_file=None) 
graph = graphviz.Source(dot_data) 
graph.render("iris")